<h2>Imports</h2>

In [ ]:
import cv2 as cv
import numpy as np
from matplotlib import pyplot as plt
from glob import glob
import copy

Error: Jupyter cannot be started. Error attempting to locate jupyter: Data Science library jupyter is not installed.

<h2>Utils

In [ ]:
def show_images(images,titles=None):
    #This function is used to show image(s) with titles by sending an array of images and an array of associated titles.
    # images[0] will be drawn with the title titles[0] if exists
    # You aren't required to understand this function, use it as-is.
    n_ims = len(images)
    if titles is None: titles = ['(%d)' % i for i in range(1,n_ims + 1)]
    fig = plt.figure()
    n = 1
    for image,title in zip(images,titles):
        a = fig.add_subplot(1,n_ims,n)
        if image.ndim == 2: 
            plt.gray()
        plt.imshow(image)
        a.set_title(title)
        n += 1
    fig.set_size_inches(np.array(fig.get_size_inches()) * n_ims)
    plt.show()


def get_nth_maximum(arr,n):
    uniqueValues = list(set(arr.flatten()))
    uniqueValues.sort()
    if len(uniqueValues) < n:
        return uniqueValues[0]
    return uniqueValues[len(uniqueValues)-n]
    # sortedmatrix.sort()
    # # print(arr.shape)
    # if sortedmatrix.size == 0:
    #     return -1
    # return sortedmatrix[0 if n > sortedmatrix.size else -n] 


<h2>Constants</h2>

In [ ]:
datasetPath = "Data/dinoSparseRing/*.png"
ß1 = 2
ß2 = 32  
# We associate with p a reference image R(p),the images S(p) where p should be visible and the images T(p) where it is truly found  
patchModel = {
    "R":None,
    "S":set,
    "T":set
}
# The cell C(i, j) keeps track of two different sets Qt(i, j) and Qf(i, j)
cell = {
    "Qt":set,
    "Qf":set
}
# We associate with each image I a regular grid of β1×β1 pixel^2 cells
imageModel = {
    "img":np.ndarray,
    "grid":np.ndarray
}

<h2>Get Input

In [ ]:
# Initialize image model from a given path
def init_imgs(datasetPath):
    # Read imgs
    imgs = [cv.rotate(cv.imread(file),cv.ROTATE_90_COUNTERCLOCKWISE) for file in glob(datasetPath)]
    # imgs = [cv.rotate(img,cv.ROTATE_90_COUNTERCLOCKWISE) for img in imgs]

    # Construct corresponding image grid
    grids = [np.array([np.array([cell for x in range(0,img.shape[1]//ß1)]) for y in range(0,img.shape[0]//ß1)]) for img in imgs]

    # print('images count:',len(imgs))
    # print('grids count:',len(grids))
    # print('image size:',imgs[0].shape)
    # print('grid size:',grids[0].shape)

    return imgs,grids

<h2>Feature Detection

In [ ]:
# Get Harris and DoG operators for a given image
def get_dog_harris(img):
    gray = cv.cvtColor(img,cv.COLOR_BGR2GRAY)
    gray = np.float32(gray)
            
    # Get DoG
    g1 = cv.GaussianBlur(gray,(0,0),sigmaX=1)
    g2 = cv.GaussianBlur(gray,(0,0),sigmaX=1*np.sqrt(2))
    diff = cv.absdiff(g1,g2)
    # dog = dog*(np.max(dog)/255)
    dog = diff * gray

    # Get Harris
    bSize = 3
    kSize = 1
    corners = cv.cornerHarris(src=gray,blockSize=bSize,ksize=kSize,k=0.06)
    # corners = cv.dilate(corners,None)

    # show_images([img,diff,dog],['detected corners blockSize='+str(bSize)+' ksize='+str(kSize),'dog before multiplication of img','dog after multiplication of img'])
    
    return dog , corners


def sparse_dog_harris(dog,harris):
    n = 4
    sparseDog = copy.copy(dog)
    sparseHarris = copy.copy(harris)
    sparseDogPositions = []
    sparseHarrisPositions = []
    for yIdx in range(0,len(dog),ß2):
        for xIdx in range(0,len(dog[0]),ß2):
            nThMaximumDog = get_nth_maximum(dog[yIdx:yIdx+ß2,xIdx:xIdx+ß2],n)
            if nThMaximumDog != -1:
                found = False
                for rowIdx,row in enumerate(dog[yIdx:yIdx+ß2]):
                    for columnIdx,column in enumerate(row[xIdx:xIdx+ß2]):
                        if not found and column == nThMaximumDog:
                            found = True
                            if column != 0:
                                sparseDogPositions.append((yIdx+rowIdx,xIdx+columnIdx))
                        else:
                            sparseDog[yIdx+rowIdx,xIdx+columnIdx] = 0
                # sparseDog[yIdx:yIdx+ß2,xIdx:xIdx+ß2] = sparseDog[yIdx:yIdx+ß2,xIdx:xIdx+ß2]*(sparseDog[yIdx:yIdx+ß2,xIdx:xIdx+ß2] == nThMaximumDog)
            nThMaximumHarris = get_nth_maximum(harris[yIdx:yIdx+ß2,xIdx:xIdx+ß2],n)
            if nThMaximumHarris != -1:
                found = False
                for rowIdx,row in enumerate(harris[yIdx:yIdx+ß2]):
                    for columnIdx,column in enumerate(row[xIdx:xIdx+ß2]):
                        if not found and column == nThMaximumHarris:
                            found = True
                            if column != 0:
                                sparseHarrisPositions.append((yIdx+rowIdx,xIdx+columnIdx))
                        else:
                            sparseHarris[yIdx+rowIdx,xIdx+columnIdx] = 0
                # sparseHarris[yIdx:yIdx+ß2,xIdx:xIdx+ß2] = sparseHarris[yIdx:yIdx+ß2,xIdx:xIdx+ß2]*(sparseHarris[yIdx:yIdx+ß2,xIdx:xIdx+ß2] == nThMaximumHarris)
            # show_images([dog[yIdx:yIdx+ß2,xIdx:xIdx+ß2],sparseDog[yIdx:yIdx+ß2,xIdx:xIdx+ß2],harris[yIdx:yIdx+ß2,xIdx:xIdx+ß2],sparseHarris[yIdx:yIdx+ß2,xIdx:xIdx+ß2]],['before dog','after dog','before harris','after harris'])

    sparseDog = cv.dilate(sparseDog,None)
    # sparseDog = cv.dilate(sparseDog,None)
    # sparseHarris = cv.dilate(sparseHarris,None)
    sparseHarris = cv.dilate(sparseHarris,None)
    return sparseDog,sparseHarris,sparseDogPositions,sparseHarrisPositions


In [ ]:
images,grids = init_imgs(datasetPath)
imagesFeatures = []
for idx,image in enumerate(images):
    dog,harris = get_harris_dog(image)
    imagesFeatures.append(
        {
            "image":images[idx],
            "grid":grids[idx],
            "dog":dog,
            "harris":harris
        }
    )

<h2>Trangulation

In [ ]:
P1 = np.float32(np.array([
    [-0.14396457836077139000, 0.96965263281337499000, 0.19760617153779569000, -0.90366580603479685000],[-0.04743335255026152200, -0.42560419233334673000, -0.40331536459778505000, -0.23984130575212276000],[0.88306936201487163000, -0.010415508744, -0.0294278883669, 0.673097816109]
]))

P2 = np.float32(np.array([
    [-0.23143687262851170000, 0.96422332027030622000, 0.12926937165558827000, -0.65860811165749811000],[-0.05749426020475298400, -0.75028640486482034000, -0.71601208187197274000, -0.25878137856376976000],[0.64835049619606955000, -0.0143392676288,-0.0315356332107, 0.659314086504]
]))

p1 = np.float32([100,6])
p2 = np.float32([900,9])

print(cv.triangulatePoints(P1,P2,p1,p2))
# F = [
#     [3310.400000, 0.000000, 316.730000],
#     [0.000000, 3325.500000, 200.550000],
#     [0.000000, 0.000000, 1.000000]
# ]
# F = np.array(F)

# p1 = (5,6,1)
# p2 = (10,9,1)

# T1 = [
#     [1,0,-p1[0]],
#     [0,1,-p1[1]],
#     [0,0,1]
# ]
# T2 = [
#     [1,0,-p2[0]],
#     [0,1,-p2[1]],
#     [0,0,1]
# ]

# T2 = np.array(T2)
# T1 = np.array(T1)
# F = np.transpose(np.linalg.inv(T2))*F*np.linalg.inv(T1)

# plt.subplot(221)
# plt.scatter([p1[0]],[p1[1]],color='red')
# plt.subplot(222)
# plt.scatter([p2[0]],[p2[1]],color='red')